**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-26 21:45:46,387 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-26 21:45:46.391 | INFO     | __main__:<module>:3 - b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-01-26 21:45:46.450 | INFO     | __main__:<module>:13 - User Role: reviewer
2025-01-26 21:45:46.452 | INFO     | __main__:<module>:24 - Creating account with name: practical_curran
2025-01-26 21:45:46.453 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 21:45:46.609 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'
2025-01-26 21:45:46.611 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:45:46.612 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 21:45:46.614 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-26 21:45:46,639 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:45:46,643 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0

b'2a42991ff2a35fabfcaa09e13267a66bd03db50a6538147ff947641f21b4a524'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-26 21:45:47.317 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 21:45:47,320 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:45:47,334 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:45:47,337 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 21:45:50,333 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 21:45:50,344 - INFO - ('COMMITTED', 5, 0)
2025-01-26 21:45:50,346 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': 'db3ac517c88700ff5446cc5de5fa08b94eccb83e0d97c3077c60851b43857652'}
2025-01-26 21:45:50.347 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 21:45:50,354 - INFO - Appended data under 'practical_curran@test' in logs/account_data.json
2025-01-26 21:45:50.357 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 21:45:50.359 | DEBUG    | int

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-26 21:45:50,397 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-26 21:45:50.399 | INFO     | __main__:<module>:3 - b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-26 21:45:50.413 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 21:45:50.462 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'
2025-01-26 21:45:50.464 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: ECAB4A3243301E6F3C6EABAC6D6DE1CE7E703118
2025-01-26 21:45:50.465 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 21:45:50.472 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-26 21:45:50,480 - INFO - Processing account: practical_curran@test
2025-01-26 21:45:50,484 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Practical Curran', 'foaf:mbox': 'practical_curran@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Universidad de Hermosillo'}, 'sc

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-01-26 21:45:53.445 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 21:45:53,451 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:45:53,471 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:45:53,476 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 21:45:56,433 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 21:45:56,441 - INFO - ('COMMITTED', 5, 0)
2025-01-26 21:45:56,443 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': '2a806396690ada7d76114223a5fb6a594f86d0842810ca125d806ab4c99353ab'}
2025-01-26 21:45:56.446 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 21:45:56,449 - INFO - Appended data under 'practical_curran@test' in logs/account_data.json
2025-01-26 21:45:56.452 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 21:45:56.454 | DEBUG    | int

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-26 21:45:56.531 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'practical_curran@test', 'schema:publicKey': 'fde634f47ffccc9d14c062d8fd55817fc422b5cb39f5a4b20ad6d24a1844ec10', 'schema:roleName': 'reviewer'}, 'foaf:mbox': 'practical_curran@email.com', 'foaf:name': 'Practical Curran', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Universidad de Hermosillo'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '2497-3197-3587-X'}}
